In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
np.random.seed(42)
x = np.random.rand(100,1) -  0.5
y =  3 * x[:, 0]**2 + 0.05 * np.random.rand(100)

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['X'] = x.reshape(100)
df['y'] = y

In [ ]:
df.head()


In [ ]:
plt.scatter(df['X'],df['y'])

Step1 : Compute the mean of target variable here is 'y'


In [ ]:
df['pred1'] = df['y'].mean()

In [ ]:
df.head()

Step2 : calculate Peseudo Residual

In [ ]:
df['res1'] = df['y'] - df['pred1']

In [ ]:
df.head(2)

In [ ]:
plt.scatter(df['X'],df['y'])
plt.plot(df['X'],df['pred1'],color='red')

Step 3: Build A Decision Tree with Help X and Pred1(Residual)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree1 =  DecisionTreeRegressor(max_leaf_nodes=8)

In [ ]:
tree1.fit(df['X'].values.reshape(100,1),df['res1'].values)

In [ ]:
from sklearn.tree import plot_tree
plot_tree(tree1)
plt.show()

Step4 : we need M1 + M2



In [ ]:
X_test = np.linspace(-0.5,0.5,500)

In [ ]:
previous_model = 0.265458

In [ ]:
y_pred = previous_model +  tree1.predict(X_test.reshape(500,1))

In [ ]:
plt.plot(X_test,y_pred, linewidth=2, color='red')
plt.scatter(df['X'],df['y'])

In [ ]:
df

In [ ]:
df['pred2'] = previous_model + tree1.predict(df['X'].values.reshape(100,1))

In [ ]:
df

In [ ]:
df['res2'] = df['y'] - df['pred2']

In [ ]:
df.head()

In [ ]:
tree2 =  DecisionTreeRegressor(max_leaf_nodes=8)

tree2.fit(df['X'].values.reshape(100,1),df['res2'].values)

In [ ]:
df['pred3'] = previous_model + tree2.predict(df['X'].values.reshape(100,1))

In [ ]:
df.head(1)

In [ ]:
df['res3'] = df['y'] - df['pred3']

In [ ]:
tree3 =  DecisionTreeRegressor(max_leaf_nodes=5)

tree3.fit(df['X'].values.reshape(100,1),df['res3'].values)

In [ ]:
y_pred1 = previous_model+sum(regressor.predict(X_test.reshape(-1, 1)) for regressor in [tree1,tree2])

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(121)
plt.plot(X_test, y_pred1, linewidth=2,color='red')
plt.scatter(df['X'],df['y'])
plt.title('X vs y')

In [ ]:
y_pred2 = previous_model+sum(regressor.predict(X_test.reshape(-1, 1)) for regressor in [tree1,tree2])

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(121)
plt.plot(X_test, y_pred2, linewidth=2,color='red')
plt.scatter(df['X'],df['y'])
plt.title('X vs y')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=13
)

params = {
    "n_estimators": 10000,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
    "verbose": 1
}

In [ ]:
reg1 = ensemble.GradientBoostingRegressor(**params)
reg1.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg1.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
# Assuming `reg` is already trained
train_predictions = reg1.predict(X_train)
train_mse = mean_squared_error(y_train, train_predictions)
train_rmse = np.sqrt(train_mse)

print("The root mean squared error (RMSE) on training set: {:.4f}".format(train_rmse))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

# Assessing Overfitting or Underfitting
if train_rmse < mse:
    print("The model is potentially underfitting.")
elif train_rmse > mse:
    print("The model is potentially overfitting.")
else:
    print("The model is balanced.")

In [ ]:
# Calculate RMSE for test set
rmse = np.sqrt(mse)
print("The root mean squared error (RMSE) on test set: {:.4f}".format(rmse))

# Calculate RMSE for training set
train_rmse = np.sqrt(train_mse)
print("The root mean squared error (RMSE) on training set: {:.4f}".format(train_rmse))

In [ ]:
test_score = np.zeros((params["n_estimators"],), dtype=np.float64)
for i, y_pred in enumerate(reg.staged_predict(X_test)):
    test_score[i] = mean_squared_error(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title("Deviance")
plt.plot(
    np.arange(params["n_estimators"]) + 1,
    reg.train_score_,
    "b-",
    label="Training Set Deviance",
)
plt.plot(
    np.arange(params["n_estimators"]) + 1, test_score, "r-", label="Test Set Deviance"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Deviance")
fig.tight_layout()
plt.show()

In [ ]:
feature_importance = reg.feature_importances_
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx], align="center")
plt.yticks(pos, np.array(diabetes.feature_names)[sorted_idx])
plt.title("Feature Importance (MDI)")

result = permutation_importance(
    reg, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(
    result.importances[sorted_idx].T,
    vert=False,
    labels=np.array(diabetes.feature_names)[sorted_idx],
)
plt.title("Permutation Importance (test set)")
fig.tight_layout()
plt.show()